In [1]:
import pandas as pd
import numpy as np
import random
from ortools.sat.python import cp_model
import names

In [2]:
def doub_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M:%S')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2])

def tri_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    nova_linha_3 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M:%S')
    nova_linha_3['Horário'] = (hora_inicial + pd.Timedelta(hours=2)).strftime('%H:%M:%S')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2, nova_linha_3])

def desaninhar_dias(df):
    df['Dias da Semana'] = df['Dias da Semana'].str.split(',')
    df = df.explode('Dias da Semana')
    return df

# Aplicar a função para todas as linhas triplas
def apply_func(df,func):
    linhas_triplicadas = df.apply(func, axis=1)
    return pd.concat(linhas_triplicadas.tolist(), ignore_index=True)

In [3]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/ROTA TESTE CARLOS E ALLAN.xlsx',header=1)
aulas_tratadas = aulas_raw.loc[~aulas_raw['Grupo'].isnull()][['Grupo','Horário','Dias da Semana','Unidade']]

In [4]:
aulas_tratadas = aulas_tratadas.loc[aulas_tratadas['Dias da Semana']!='FOLDER'].reset_index(drop=True)

In [5]:
aulas = aulas_tratadas.copy()
# tratando os dados para colocar cada linha uma aula
aulas['Dias da Semana'] = aulas['Dias da Semana'].str.replace('EVERYDAY','2ª ● 3ª ● 4ª ● 5ª ● 6ª')

# separando as aulas que são triplas, duplas e o resto
tri = aulas.loc[aulas['Dias da Semana'] == 'Saturday - Triple']
doub = aulas.loc[aulas['Dias da Semana'].str.contains('DOUBLE')]
aulas_simples = aulas[~aulas['Dias da Semana'].str.contains('DOUBLE|Saturday - Triple')].copy()

# tratando a colunas horario
aulas_simples['Horário'] = pd.to_datetime(aulas_simples['Horário'],format='%H:%M:%S').dt.strftime('%H:%M:%S')

In [6]:
# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = apply_func(doub, doub_row)
aulas_triplicadas = apply_func(tri, tri_row)

# padronizando a coluna de dias da semana
aulas_duplicadas['Dias da Semana'] = aulas_duplicadas['Dias da Semana'].str.replace('DOUBLE','')
aulas_triplicadas['Dias da Semana'] = aulas_triplicadas['Dias da Semana'].str.replace(' - Triple','')

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

In [7]:
# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = apply_func(doub, doub_row)
aulas_triplicadas = apply_func(tri, tri_row)

# padronizando a coluna de dias da semana
aulas_duplicadas['Dias da Semana'] = aulas_duplicadas['Dias da Semana'].str.replace('DOUBLE','')
aulas_triplicadas['Dias da Semana'] = aulas_triplicadas['Dias da Semana'].str.replace(' - Triple','')

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

# desaninhando os dias da semana
df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.replace('●',',')

In [8]:
# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = apply_func(doub, doub_row)
aulas_triplicadas = apply_func(tri, tri_row)

# padronizando a coluna de dias da semana
aulas_duplicadas['Dias da Semana'] = aulas_duplicadas['Dias da Semana'].str.replace('DOUBLE','')
aulas_triplicadas['Dias da Semana'] = aulas_triplicadas['Dias da Semana'].str.replace(' - Triple','')

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

# desaninhando os dias da semana
df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.replace('●',',').str.replace(' ','').str.split(',')
df_tratado = df_tratado.explode('Dias da Semana').reset_index(drop=True)

df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.replace('ª','ª,').str.split(',')
df_tratado = df_tratado.explode('Dias da Semana').reset_index(drop=True)

df_tratado = df_tratado[df_tratado['Dias da Semana']!=''].copy()


In [9]:
nomes_distintos = set()  # Usar um conjunto para garantir a distinção
tem_carro = np.random.choice([0, 1], size=50, p=[0.3, 0.7])
while len(nomes_distintos) < 50:
    primeiro_nome = names.get_first_name()
    nomes_distintos.add(primeiro_nome)

professors = list(nomes_distintos)
has_car = list(tem_carro)
info_professors = pd.DataFrame({'professor': professors, 'tem_carro': has_car})
info_professors.head()

,professor,tem_carro
0,Douglas,1
1,Wesley,1
2,Marcia,1
3,Rudolf,0
4,Teresa,1


In [10]:
# Dicionário de alocações
alocacoes = {}

# Modelo
model = cp_model.CpModel()

for i in professors:
    for g in df_tratado['Grupo'].unique():
        alocacoes[(i,g)] = model.NewBoolVar(f"{i}_converinglesson_{g}")

#### condição : só um professor por grupo

In [11]:
for g in df_tratado['Grupo'].unique():
    model.Add(sum(alocacoes[(i,g)] for i in professors) == 1)

#### condição : professor só da aula em uma turma de mesmo horario

In [12]:
horarios = df_tratado['Horário'].unique()
dias = df_tratado['Dias da Semana'].unique()
for h in horarios:
    for d in dias: 
        grupos_no_mesmo_horario = df_tratado.loc[((df_tratado['Horário'] == h) & (df_tratado['Dias da Semana'] == d))]['Grupo'].unique()
        for i in professors:
            model.Add(sum(alocacoes[(i,g)] for g in grupos_no_mesmo_horario) <= 1)

#### codição : professor que nao tem carro nao pode dar aulas seguidas em regioes distintas

In [74]:
for i in df_tratado['Dias da Semana'].unique():
    for j in df_tratado['Grupo'].unique():
        if df_tratado.loc[(df_tratado['Grupo'] == j) & (df_tratado['Dias da Semana'] == i)]['Horário'].empty:
            continue
        horario_da_turma = df_tratado.loc[(df_tratado['Grupo'] == j) & (df_tratado['Dias da Semana'] == i)]['horario_tratado'].values[0]
        grupos_seguidos = df_tratado.loc[(df_tratado['Dias da Semana'] == i) & (df_tratado['Horário'] == (horario_da_turma+ pd.Timedelta(hours=1)).strftime('%H:%M:%S'))]['Grupo'].unique()
        if len(grupos_seguidos) > 0:
            print(f"Esse grupo:{j} tem que ser seguido por {grupos_seguidos}")
        

Esse grupo:MBA DAVI BRAGA tem que ser seguido por ['SEATTLE PRESENCIAL']
Esse grupo:MBA IDEIAS EM AÇÃO tem que ser seguido por ['CAPE COD PRESENCIAL' 'TABLEDERRY ONLINE' 'DANILO - ONLINE'
 'BIG BEAR LAKE ONLINE' 'BAKERSFIELD PRESENCIAL' 'TOOWOOMBA ONLINE'
 'RONALDO - ONLINE' 'MILWAUKEE ONLINE' 'QUEENSLAND ONLINE'
 'NOVA SCOTIA ONLINE' 'WISCONSIN ONLINE' 'MILFORD ONLINE'
 'HOLLYWOOD PRESENCIAL' 'JAMAICA ONLINE ' 'BOULDER PRESENCIAL'
 'ERIK - PRESENCIAL' 'CONVERSATION 5 ONLINE' 'LEOMINSTER ONLINE'
 'QUEBEC PRESENCIAL' 'VALENCIA - ONLINE' 'INTENSIVÃO 3 ONLINE']
Esse grupo:GREENSBURG ONLINE tem que ser seguido por ['FERNANDO CAJE - PRESENCIAL' 'KNOXVILLE PRESENCIAL' 'CURAÇAU ONLINE'
 'MEMPHIS PRESENCIAL' 'IOWA ONLINE' 'CHARLOTTE PRESENCIAL'
 'SAN FRANCISCO ONLINE' 'ERILENE - PRESENCIAL/ONLINE']
Esse grupo:SAVANAH PRESENCIAL tem que ser seguido por ['FERNANDO CAJE - PRESENCIAL' 'KNOXVILLE PRESENCIAL' 'CURAÇAU ONLINE'
 'MEMPHIS PRESENCIAL' 'IOWA ONLINE' 'CHARLOTTE PRESENCIAL'
 'SAN FRANCISCO

In [13]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
prof_alocados = pd.DataFrame(columns=['Professor', 'Grupo'])

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for g in df_tratado['Grupo'].unique():
        for i in professors:
            if solver.Value(alocacoes[(i, g)]):
                print(f"O professor {i} foi alocado ao grupo {g}")
                aloca = pd.DataFrame({'Professor': [i], 'Grupo': [g]})
                prof_alocados = pd.concat([prof_alocados, aloca], ignore_index=True)

else:
    print("Nenhuma solução viável foi encontrada.")

O professor Alvin foi alocado ao grupo MBA DAVI BRAGA
O professor Antonio foi alocado ao grupo MBA LIGA DOS CAMPEÕES
O professor Antonio foi alocado ao grupo MBA IDEIAS EM AÇÃO
O professor Antonio foi alocado ao grupo GREENSBURG ONLINE
O professor Virginia foi alocado ao grupo SAVANAH PRESENCIAL
O professor Antonio foi alocado ao grupo CONVERSATION 14 PRESENCIAL
O professor Virginia foi alocado ao grupo FERNANDO CAJE - PRESENCIAL
O professor Alvin foi alocado ao grupo KNOXVILLE PRESENCIAL
O professor Virginia foi alocado ao grupo GREAT BRITAIN PRESENCIAL
O professor Jane foi alocado ao grupo GARLAND PRESENCIAL
O professor Tara foi alocado ao grupo TAMPA PRESENCIAL
O professor Antonio foi alocado ao grupo CAPE COD PRESENCIAL
O professor Alvin foi alocado ao grupo TABLEDERRY ONLINE
O professor Melissa foi alocado ao grupo ANA PAULA - ONLINE
O professor Alvin foi alocado ao grupo KENTUCKY ONLINE
O professor Melissa foi alocado ao grupo CURAÇAU ONLINE
O professor Jimmie foi alocado ao grup

In [14]:
aulas_finais = aulas_tratadas.merge(prof_alocados, on='Grupo', how='left')

In [15]:
aulas_finais['Professor'].nunique()

29

In [16]:
aulas_finais

,Grupo,Horário,Dias da Semana,Professor
0,MBA DAVI BRAGA,14:00:00,2ª,Alvin
1,MBA LIGA DOS CAMPEÕES,18:30:00,5ª,Antonio
2,MBA IDEIAS EM AÇÃO,19:00:00,2ª,Antonio
3,GREENSBURG ONLINE,07:00:00,2ª ● 3ª ● 4ª ● 5ª,Antonio
4,INTENSIVÃO 3 ONLINE,19:00:00,DOUBLE EVERYDAY,Jacob
...,...,...,...,...
142,AUCKLAND PRESENCIAL,19:00:00,2ª ● 3ª ● 4ª ● 5ª,Curtis
143,QUEBEC PRESENCIAL,20:00:00,2ª ● 4ª ● 5ª,Georgia
144,GUADALAJARA ONLINE,19:00:00,2ª ● 3ª ● 4ª ● 5ª,Jennifer
145,VALENCIA - ONLINE,20:00:00,2ª ● 3ª ● 4ª ● 5ª,Eleanor
